In [3]:
class APAlala:
    def sec_paper(pagina_web, user_agent):
        import urllib
        from bs4 import BeautifulSoup
        import pandas as pd
        import numpy as np
        from urllib.error import HTTPError
        arr_ppr = []
        headers = {}
        headers['User-Agent'] = user_agent
        page = (pagina_web)
        page_paper = urllib.request.Request(page, headers=headers)
        page_paper_req = urllib.request.urlopen(page_paper).read().decode('utf-8')
        soup_ppr = BeautifulSoup(page_paper_req, 'html.parser')
        ul_class_ppr=soup_ppr.find_all('ul')
        for i in range(len(ul_class_ppr)):
            dir_sppr = ul_class_ppr[i]
            for link in dir_sppr.find_all('a'):
                arr_ppr = np.append(arr_ppr, link.get('href'))
    
        arr_pp_app = []
        arr_pp_s = pd.Series(arr_ppr)
        arr_pp_s.dropna(inplace=True)
        arr_pp_s.reset_index(drop=True, inplace=True)
       
        for i in range(len(arr_pp_s)):
            arr_split = arr_pp_s[i].split(pagina_web)[-1]
            arr_pp_app = np.append(arr_pp_app, arr_split)
         
        dir_tabl = pd.DataFrame({'Secciones' : arr_pp_app,
                                 'Direcciones' : arr_pp_s})

        return print('Las secciones del diario son:', np.unique(arr_pp_app))
    
    def argie_panalyzer(pagina_web, seccion, user_agent):
        import urllib
        from bs4 import BeautifulSoup
        import pandas as pd
        import numpy as np
        from urllib.error import HTTPError
        import re
        from legibilidad import legibilidad
        import nltk
        import string
        from collections import Counter
        nltk.download('punkt')
        nltk.download('stopwords')
        
        pagina_web = pagina_web
        seccion = seccion
        user_agent = user_agent
        cant_art = 0
        arr_ppr = []
        headers = {}
        headers['User-Agent'] = user_agent
        page = (pagina_web)
        if False:
            try:
                page_paper = urllib.request.Request(page, headers=headers)
                page_paper_req = urllib.request.urlopen(page_paper).read().decode('utf-8')
                soup_ppr = BeautifulSoup(page_paper_req, 'html.parser')
                ul_class_ppr=soup_ppr.find_all('ul')
                dir_sppr = ul_class_ppr[0]
                for link in dir_sppr.find_all('a'):
                    arr_ppr = np.append(arr_ppr, link.get('href'))
                arr_pp_app = []
                arr_pp_s = pd.Series(arr_ppr)
                arr_pp_s.dropna(inplace=True)
                arr_pp_s.reset_index(drop=True, inplace=True)
                for i in range(len(arr_pp_s)):
                    arr_split = arr_pp_s[i].split('/')
                    while '' in arr_split:
                        arr_split.remove('')
                    ultimo = arr_split[-1]
                    arr_pp_app = np.append(arr_pp_app, ultimo)
                dir_tabl = pd.DataFrame({'Secciones' : arr_pp_app,
                                         'Direcciones' : arr_pp_s})
            except IndexError:
                    pass
        
        else:
            page_paper = urllib.request.Request(page, headers=headers)
            page_paper_req = urllib.request.urlopen(page_paper).read().decode('utf-8')
            soup_ppr = BeautifulSoup(page_paper_req, 'html.parser')
            ul_class_ppr=soup_ppr.find_all('ul')
            dir_sppr = ul_class_ppr[2]
            for link in dir_sppr.find_all('a'):
                arr_ppr = np.append(arr_ppr, link.get('href'))
            arr_pp_app = []
            arr_pp_s = pd.Series(arr_ppr)
            arr_pp_s.dropna(inplace=True)
            arr_pp_s.reset_index(drop=True, inplace=True)
            for i in range(len(arr_pp_s)):
                arr_split = arr_pp_s[i].split('/')
                while '' in arr_split:
                    arr_split.remove('')
                ultimo = arr_split[-1]
                arr_pp_app = np.append(arr_pp_app, ultimo)
            dir_tabl = pd.DataFrame({'Secciones' : arr_pp_app,
                                     'Direcciones' : arr_pp_s})
  
        sec = seccion
        for i in range(len(dir_tabl['Direcciones'])):
            if sec in dir_tabl['Secciones'][i]:
                sec = dir_tabl['Direcciones'][i]
                break

        item = pd.Series([page]+[sec])
        if item[0] not in item[1]:
            res_web=(pd.Series([page]+[sec])).sum()
        else:
            res_web=item[1]    
    
        headers = {}
        headers['User-Agent'] = user_agent
        html = res_web
        url = urllib.request.Request(html, headers=headers)
        url = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(url, 'html.parser')
        dir_url = []
        for i in soup.find_all('a'):
            dir_url= np.append(dir_url, i.get('href'))
        dir_url = pd.Series(dir_url)
        paginas = pd.DataFrame({})
        paginas['Direcciones'] = dir_url
        paginas.dropna(inplace=True)
        paginas.reset_index(drop=True, inplace=True)
    
        urls=[]
        for sec in np.arange(len(paginas['Direcciones'])):
            urls = np.append(urls, paginas['Direcciones'].iloc[sec])
    
        urls = np.unique(urls)
        urls_res = []
        for i in range(len(urls)):
            if re.search(r"ar/+[0-9]", urls[i]):
                urls_res = np.append(urls_res, urls[i])
            elif '/'+seccion+'/' == urls[i]:
                np.delete(urls, [i])
            elif '/'+seccion == urls[i]:
                np.delete(urls, [i])
            elif re.search(r"^/[0-9]+-", urls[i]):
                urls_res = np.append(urls_res, urls[i])
            elif re.search(r'^'+seccion, urls[i]):
                urls_res = np.append(urls_res, urls[i])
            elif re.search(r'^/'+seccion, urls[i]):
                urls_res = np.append(urls_res, urls[i])
    
        todo_texto = []
        diff_text= []
        for i in np.arange(len(urls_res)):
            try:
                if pagina_web in urls_res[i]:
                    busqueda = urls_res[i]
                elif seccion == urls_res[i]:
                    continue
                else:
                    busqueda = pagina_web+urls_res[i]
                html_def = busqueda
                url_def = urllib.request.Request(html_def, headers=headers)
                url_def = urllib.request.urlopen(url_def).read()
                soup_def = BeautifulSoup(url_def)
                titulo_def = soup_def.find('title')
                cant_art = cant_art +1
                articulo_def = soup_def.find_all('p')
                art_def_text=[re.sub(r'\<.+?>',r'',str(a)) for a in articulo_def]
                cuenta= []
                for i in range(len(art_def_text)):
                    cuenta = np.append(cuenta, len(art_def_text[i]))
                defin = []
                for i in np.arange(len(art_def_text)):
                    if len(art_def_text[i])>(cuenta.mean()+40):
                        defin= np.append(defin, art_def_text[i])
                art_def_text = (pd.Series(defin)).sum()
                art_def_text = art_def_text.replace('\xa0', ' ')
                if 'Recibir newsletter' in art_def_text:
                    art_def_text = art_def_text.replace('Recibir newsletter', ' ')
                ln_comentario = 'Los comentarios publicados son de exclusiva responsabilidad de sus autores y las consecuencias derivadas de ellos pueden ser pasibles de sanciones legales. Aquel usuario que incluya en sus mensajes algún comentario violatorio del reglamento será eliminado e inhabilitado para volver a comentar. Enviar un comentario implica la aceptación del Reglamento.'
                if ln_comentario in art_def_text:
                    art_def_text = art_def_text.replace(ln_comentario, '.')
                if '.' in art_def_text:
                    if '.' != '. ':
                        art_def_text = art_def_text.replace('.', '. ')
            except (AttributeError, HTTPError):
                print('Imposible ejecutar sobre', busqueda, '\n')
                cant_art = cant_art - 1
                continue
            
            todo_texto = np.append(todo_texto, art_def_text)
            print('Título', titulo_def, '\nDificultad de lectura (szigrizt_pazos):', legibilidad.interpretaP(legibilidad.szigriszt_pazos(art_def_text)), '\n')
            diff_text = np.append(diff_text, legibilidad.szigriszt_pazos(art_def_text))

        print('\nPromedio de dificultad de lectura:', legibilidad.interpretaP(np.average(diff_text)), '(',np.average(diff_text),')\n\n')
        print('Cantidad de artículos analizados en', seccion,':', cant_art, '\n')

        todo_texto = (pd.Series(todo_texto)).sum()   
        texto_words = nltk.word_tokenize(todo_texto.lower())
        {word:True for word in texto_words}
        def build_bag_of_words_features(words):
            return {word:True for word in words}
        lista = ['¿' ,'¡', '""', "''",'“', '”', '``', '-', '«', '»']
        lista2 = ['así', 'si', 'sólo', 'modo', 'ción', 'unas', 'puede', 'menos', 'punto', 'caso', 'según', 'todas', 'ciento', 'vez', 'tal', 'condiciones©', 'pagina12', 'www', 'com', 'comment', 'clarín', 'clic', 'comentar', 'e-mail', '--']
        useless_words = nltk.corpus.stopwords.words("spanish") + list(string.punctuation) + list(string.digits) + lista + lista2
        filtered_words = [word for word in texto_words if not word in useless_words]
        word_counter = Counter(filtered_words)
        most_common_words = word_counter.most_common()[:]
    
        return print(most_common_words[:60])

In [4]:
APAlala.sec_paper('https://www.clarin.com/', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.96 Safari/537.36')

Las secciones del diario son: ['#' '/cartas_al_pais/' '/ciudades/' '/contactenos.html' '/deportes/'
 '/economia/' '/economia/divisas-acciones-bonos/' '/espectaculos/'
 '/mundo/' '/opinion/' '/policiales/' '/politica/' '/sociedad/' 'arq/'
 'autos/' 'br/' 'buena-vida/' 'claringrilla/' 'clima/' 'cultura/'
 'deportes/' 'deportes/agenda-deportiva/' 'deportes/ascenso/'
 'deportes/basquet/liga-nacional/' 'deportes/estadisticas.html'
 'deportes/futbol-internacional/' 'deportes/futbol/' 'deportes/hockey/'
 'deportes/rugby/' 'deportes/seleccion-nacional/' 'deportes/tenis/'
 'ediciones-anteriores/' 'entremujeres/' 'especiales-clarin/'
 'espectaculos/' 'espectaculos/cine/' 'espectaculos/cine/cartelera.html'
 'espectaculos/fama/' 'espectaculos/musica/' 'espectaculos/teatro/'
 'espectaculos/teatro/cartelera.html' 'espectaculos/tv/' 'feriados/'
 'fotogalerias' 'horoscopo.html' 'http://365.clarin.com'
 'http://ar.cienradios.com/' 'http://autogestion.365.com.ar'
 'http://comercial.agea.com.ar/' 'http:/

In [6]:
APAlala.argie_panalyzer('https://www.clarin.com/','/economia/', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.96 Safari/537.36')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mauricio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mauricio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Título <title>La Ciudad colocó entre inversores privados un bono en pesos para cubrir las necesidades de 2019 - 13/02/2019 - Clarín.com</title> 
Dificultad de lectura (szigrizt_pazos): normal 

Título <title>Confirman multa contra un supermercado por no mostrar precios en góndola - 14/02/2019 - Clarín.com</title> 
Dificultad de lectura (szigrizt_pazos): bastante fácil 

Imposible ejecutar sobre https://www.clarin.com//economia/divisas-acciones-bonos/ 

Título <title>Dólar hoy: a cuánto cerró, banco por banco - 14/02/2019 - Clarín.com</title> 
Dificultad de lectura (szigrizt_pazos): fácil 

Título <title>Dólar hoy: a cuánto cerró, banco por banco - 13/02/2019 - Clarín.com</title> 
Dificultad de lectura (szigrizt_pazos): fácil 

Título <title>Dólar hoy: vuelve a subir y queda dentro de la banda cambiaria - 14/02/2019 - Clarín.com</title> 
Dificultad de lectura (szigrizt_pazos): bastante fácil 

Título <title>Tras el alza de la inflación suben el dólar, la tasa y el riesgo país - 15/02/20